In [1]:
 ## Deep Learning Sentiment Analysis
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from keras.models import Sequential, Model
from keras.layers import Embedding, Dropout, Reshape, Activation, Input, Flatten, Dense, RepeatVector, BatchNormalization, LSTM, RepeatVector, MaxPooling1D,GlobalMaxPool1D, Conv1D, GRU, Bidirectional
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
from keras.callbacks import EarlyStopping
from tqdm import tqdm
import tensorflow as tf
import joblib
import os
import requests 
import numpy as np
from pathlib import Path
print('GPU avialible: {}'.format(tf.test.is_gpu_available()))
pd.options.display.width = 500

Using TensorFlow backend.
/home/matthew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/matthew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/matthew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/matthew/.local/lib/python3.6/site-packages/tensorflow/py

GPU avialible: False


/home/matthew/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/matthew/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/matthew/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/matthew/.local/lib/python3.6/site-packages/tensorboard/compat

In [21]:


# Download Cornal Moview revirew data sets
url = 'http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz'
file_path = 'polarity.tar.gz'
path = Path(file_path)
response = requests.get(url)

# write contents of the download 
path.write_bytes(response.content)

# unziped the file create a directory called txt_sentoken
os.system('tar -xzf {}'.format(file_path))

# Note if you don't have tar, you may need to use 7zip or another unzipping tool (windows)
# set positive and negative directory paths
pos_dir = 'txt_sentoken/pos'
neg_dir = 'txt_sentoken/neg'

# define a function to read in text
def read_file_list(file_list):
    for file in file_list:
        with open(file, 'r') as f:
            text = f.readlines()
            text = ' '.join(text)
            yield text
            
# get the list of file paths
pos_files = os.listdir(pos_dir)
neg_files = os.listdir(neg_dir)


# data the working dir to the file paths
pos_files = [pos_dir + '/' + f for f in pos_files if 'cv' in f]
neg_files = [neg_dir + '/' + f for f in neg_files if 'cv' in f]
neg_reviews = list(read_file_list(neg_files))
pos_reviews = list(read_file_list(pos_files))
data = pos_reviews + neg_reviews

# positive 1, negative review 5
labels = np.array([1] * len(pos_reviews) + [0] * len(neg_reviews))

#
print(data[1][:200])
print('label:', labels[1])
print('n_docs {}'.format(len(data)))

bruce willis and sixth sense director m . night shyamalan re-team to tell the story of david dunne ( willis ) , a stadium security guard who has been having some problems at home that are affecting hi
label: 1
n_docs 2000


In [3]:
# sets the max number of words in the tokenizer
vocab_size = 1000
max_len = 1000
tokenizer = Tokenizer(vocab_size)

# fits on tokenizer
tokenizer.fit_on_texts(data)

print('vocab_size {0}, max_len {1}'.format(vocab_size, max_len))

vocab_size 1000, max_len 1000


In [4]:
## load embeddings with 100 vector inputs
path_to_glove_embeddings = '/home/matthew/repos/glove/glove.6B.100d.txt'
embeddings_index = {}

# loads glove embeddings ()
f = open(path_to_glove_embeddings, encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [5]:
# create embedding weight array (looks up words from the tokenizers vocab)
dim_embedddings = 100
not_found = []
def get_emb(word_index):
    found = 0
    missed = 0
    for word, i in word_index.items():
        if i <= vocab_size:
            try:
                emb = embeddings_index[word]
            except KeyError:
                emb = np.zeros(dim_embedddings)
                not_found.append(word)
            yield emb

# adds an array of zeros for when word index is 0
embedding_weights = np.array([np.zeros(dim_embedddings)] + list(get_emb(tokenizer.word_index)))
print('embedding_weights shape {}'.format(embedding_weights.shape))
print('{}% not found'.format(round(len(not_found)/vocab_size *100,2)))

embedding_weights shape (1001, 100)
2.9% not found


In [17]:
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [22]:
X_train = tokenizer.texts_to_sequences(docs_train)
X_train = np.array(pad_sequences(X_train, padding='post', maxlen=max_len, truncating='post'))
X_test = tokenizer.texts_to_sequences(docs_test)
X_test = np.array(pad_sequences(X_test, padding='post', maxlen=max_len, truncating='post'))  
print(X_train.shape, X_test.shape)

(1340, 1000) (660, 1000)


In [28]:
def get_model():
    embedding_layer = Embedding(vocab_size + 1,dim_embedddings, weights=[embedding_weights ],
                                input_length=max_len,
                                trainable=False)

    model = Sequential()
    model.add(embedding_layer)

    model.add(GRU(64, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))

    model.add(Dense(1, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model
model = get_model()
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1000, 100)         100100    
_________________________________________________________________
gru_11 (GRU)                 (None, 64)                31680     
_________________________________________________________________
batch_normalization_11 (Batc (None, 64)                256       
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 33        
Total params: 134,149
Trainable params: 33,921
Non-trainable params: 100,228
___________________________________________

In [29]:
esm =  EarlyStopping(patience=2, monitor='loss')
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, callbacks=[esm])

Train on 1340 samples, validate on 660 samples
Epoch 1/4
1340/1340 [==============================] - 15s 11ms/step - loss: 7.6094 - accuracy: 0.5037 - val_loss: 7.7828 - val_accuracy: 0.4924
Epoch 2/4
1340/1340 [==============================] - 17s 13ms/step - loss: 7.6094 - accuracy: 0.5037 - val_loss: 7.7828 - val_accuracy: 0.4924
Epoch 3/4
1340/1340 [==============================] - 18s 13ms/step - loss: 7.6094 - accuracy: 0.5037 - val_loss: 7.7828 - val_accuracy: 0.4924
Epoch 4/4
1340/1340 [==============================] - 17s 13ms/step - loss: 7.6094 - accuracy: 0.5037 - val_loss: 7.7828 - val_accuracy: 0.4924


In [16]:
##optional method for training the model on a generator
def data_gen(docs_train, docs_test,  y_train, y_test, n_batches=1,  batch_size=32):
    for i in range(n_batches):
        docs_train = np.array(docs_train)
        docs_test = np.array(docs_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        
        # randomly shuffels training  set
        train_index = np.random.choice(np.arange(len(docs_train)), batch_size)
        X_train = tokenizer.texts_to_sequences(docs_train[train_index])
        X_train = np.array(pad_sequences(X_train, padding='post', maxlen=max_len, truncating='post'))
        
        #radomly gets the test set
        test_index = np.random.choice(np.arange(len(docs_test)), batch_size)
        X_test = tokenizer.texts_to_sequences(docs_test[test_index])
        X_test = np.array(pad_sequences(X_test, padding='post', maxlen=max_len, truncating='post'))
        yield X_train, X_test, y_train[train_index], y_test[test_index]

# test data gen
X, X_t, y, y_t = next(data_gen(docs_train, docs_test, y_train, y_test, n_batches=2, batch_size=32))
print(X.shape, X_t.shape, y.shape, y_t.shape)

# training loop
def train(n_batches = 20, batch_size = 1024):
    g  = data_gen(docs_train, docs_test, y_train, y_test, n_batches=n_batches, batch_size=batch_size)
    for i in range(n_batches):
        X, X_t, y, y_t = next(g)
        model.train_on_batch(X,y)
        print(model.evaluate(X_t, y_t))
 

1024/1024 [==============================] - 4s 4ms/step
[8.14578327536583, 0.46875]
1024/1024 [==============================] - 5s 5ms/step
[7.6067240834236145, 0.50390625]
1024/1024 [==============================] - 5s 5ms/step
[7.621697887778282, 0.5029296875]
1024/1024 [==============================] - 5s 5ms/step
[7.861279785633087, 0.4873046875]
1024/1024 [==============================] - 5s 5ms/step
[7.561802372336388, 0.5068359375]
1024/1024 [==============================] - 5s 5ms/step
[7.711541011929512, 0.4970703125]
1024/1024 [==============================] - 5s 5ms/step
[7.966096758842468, 0.48046875]
1024/1024 [==============================] - 5s 5ms/step
[7.711541101336479, 0.4970703125]


KeyboardInterrupt: 